## Daily Code Workout Day 2(220426)
1. Monty Hall Problem
2. Covid Simulation(<- Here!)

#### 00. Packages Used

In [2]:
import pandas as pd
import numpy as np
import random

#### 01.  Randomized Locations & Status

In [1]:
st_dict ={1:"sick", 0:"healthy", 9:"dead", 2:"immunized", 3:'vaccinated'}

In [108]:
ppl_n, i_ratio = 10000, 0.004
loc_df = pd.DataFrame(np.array([np.random.uniform(0, 1, ppl_n),
                                np.random.uniform(0, 1, ppl_n)])).T
loc_df.columns = ['x', 'y']
covid = random.sample(range(ppl_n), max(1, int(ppl_n*i_ratio)))
loc_df['status_0'] = np.where(loc_df.index.isin(covid), 1, 0)

#### 02 - 1. Contact & Catching Covid - Brute Force Method(Baseline, prob. not optimal)

In [171]:
het_df = loc_df[loc_df['status_0'] == 0].copy()
x_df = loc_df[loc_df['status_0'] == 1][['x']].T.reset_index(drop = True)
x_df.columns = [str(x) + "_x" for x in list(x_df)]
y_df = loc_df[loc_df['status_0'] == 1][['y']].T.reset_index(drop = True)
y_df.columns = [str(x) + "_y" for x in list(y_df)]

het_df = pd.concat([het_df, 
                    pd.DataFrame(x_df, index = het_df.index).fillna(method = "ffill"),
                    pd.DataFrame(y_df, index = het_df.index).fillna(method = "ffill")], axis  =1)

het_df = pd.concat([het_df[['x'] + list(x_df)].apply(lambda x: abs(x -x['x']), axis = 1 ),
                    het_df[['y'] + list(y_df)].apply(lambda x: abs(x -x['y']), axis = 1 )], axis = 1).drop(['x','y'], axis = 1)
het_df = np.sqrt(pd.concat([pd.DataFrame(het_df[[x for x in list(het_df) if str(y) in x]
                                               ].apply(lambda x: x[str(y) + "_x"]**2 + x[str(y) + "_y"]**2 ,axis = 1 ) )
                            for y in loc_df[loc_df['status_0'] == 1].index], axis = 1))
het_df['min'] = het_df.min(axis =1)

new_df = loc_df.copy()
new_df['status_1'] = np.where(new_df.index.isin(het_df[het_df['min'] < 0.03].index), 1,
                             new_df['status_0'])

#### 02 - 2. Function

In [185]:
def step_contact(loc_df, st_col = 'status_0'):
    s_df = loc_df[loc_df[st_col] == 1].copy()
    h_df = loc_df[loc_df[st_col] == 0].copy()
    x_df, y_df = s_df[['x']].T.reset_index(drop = True), s_df[['y']].T.reset_index(drop = True)
    x_df.columns = [str(x) + "_x" for x in list(x_df)]
    y_df.columns = [str(x) + "_y" for x in list(y_df)]

    h_df = pd.concat([h_df, 
                      pd.DataFrame(x_df, index = h_df.index),
                      pd.DataFrame(y_df, index = h_df.index)], axis = 1).fillna(method = "ffill")

    h_df = pd.concat([h_df[['x'] + list(x_df)].apply(lambda x: x - x['x'], axis = 1 ),
                      h_df[['y'] + list(y_df)].apply(lambda x: x - x['y'], axis = 1 )], axis = 1)
    h_df = h_df.drop(['x','y'], axis = 1)
    h_df = np.sqrt(pd.concat([pd.DataFrame(h_df[[x for x in list(h_df) if str(y) in x]
                                               ].apply(lambda x: x[str(y) + "_x"]**2 + x[str(y) + "_y"]**2 ,axis = 1 ) )
                              for y in s_df.index], axis = 1))
    h_df['min'] = h_df.min(axis =1)

    new_df = loc_df.copy()
    new_col = "status_" + str(int(st_col.split("_")[1]) + 1)
    new_df[new_col] = np.where(new_df.index.isin(h_df[h_df['min'] < 0.03].index), 1,
                                  new_df[st_col])
    return(new_df[['x', 'y', new_col]])

In [186]:
%%time
next_step = step_contact(loc_df, 'status_0')

Wall time: 3.61 s


## Daily Coding Workout - Day 3(220427)
1. Monty Hall Problem
2. Covid Simulation(<- Here!)

#### 02 - 3. Contact & Catching Covid - Dividing by Sectors

In [13]:
from itertools import product

In [137]:
%%time
loc_df['sect'] = loc_df.apply(lambda x: (int(x['x'] / (1/5)), int(x['y'] / (1/5))), axis = 1)
all_list = []
for sect in set(loc_df[loc_df['status_0'] == 1]['sect']):
    o_sect = [x for x in set(loc_df['sect']) if 
              np.sqrt((x[0]-sect[0])**2 + (x[1]-sect[1])**2) <= np.sqrt(2)]
    het_df = loc_df[(loc_df['status_0'] == 0) & 
                    (loc_df['sect'].isin(o_sect))].copy()
    x_df = loc_df[(loc_df['status_0'] == 1) &
                  (loc_df['sect'] == sect)][['x']].T.reset_index(drop = True)
    x_df.columns = [str(x) + "_x" for x in list(x_df)]
    x_df.index = het_df.index[:1]
    y_df = loc_df[(loc_df['status_0'] == 1) &
                  (loc_df['sect'] == sect)][['y']].T.reset_index(drop = True)
    y_df.columns = [str(x) + "_y" for x in list(y_df)]
    y_df.index = het_df.index[:1]
    
    het_df = pd.concat([het_df, 
                        pd.DataFrame(x_df, index = het_df.index).fillna(method = "ffill"),
                        pd.DataFrame(y_df, index = het_df.index).fillna(method = "ffill")], axis  =1)

    het_df = pd.concat([het_df[['x'] + list(x_df)].apply(lambda x: abs(x -x['x']), axis = 1 ),
                        het_df[['y'] + list(y_df)].apply(lambda x: abs(x -x['y']), axis = 1 )], axis = 1).drop(['x','y'], axis = 1)
    het_df = np.sqrt(pd.concat([pd.DataFrame(het_df[[x for x in list(het_df) if str(y) in x]
                                                   ].apply(lambda x: x[str(y) + "_x"]**2 + x[str(y) + "_y"]**2 ,axis = 1 ) )
                                for y in loc_df[(loc_df['status_0'] == 1) &
                                                (loc_df['sect'] == sect)].index], axis = 1))
    het_df['min'] = het_df.min(axis =1)
    all_list = all_list + list(het_df[het_df['min'] < 0.03].index)
new_df = loc_df.copy()
new_df['status_1'] = np.where(new_df.index.isin(all_list), 1, new_df['status_0'])
new_df[new_df['status_1']==1].shape

Wall time: 29 s


(1089, 5)

In [138]:
%%time
het_df = loc_df[loc_df['status_0'] == 0].copy()
x_df = loc_df[loc_df['status_0'] == 1][['x']].T.reset_index(drop = True)
x_df.columns = [str(x) + "_x" for x in list(x_df)]
y_df = loc_df[loc_df['status_0'] == 1][['y']].T.reset_index(drop = True)
y_df.columns = [str(x) + "_y" for x in list(y_df)]

het_df = pd.concat([het_df, 
                    pd.DataFrame(x_df, index = het_df.index).fillna(method = "ffill"),
                    pd.DataFrame(y_df, index = het_df.index).fillna(method = "ffill")], axis  =1)

het_df = pd.concat([het_df[['x'] + list(x_df)].apply(lambda x: abs(x -x['x']), axis = 1 ),
                    het_df[['y'] + list(y_df)].apply(lambda x: abs(x -x['y']), axis = 1 )], axis = 1).drop(['x','y'], axis = 1)
het_df = np.sqrt(pd.concat([pd.DataFrame(het_df[[x for x in list(het_df) if str(y) in x]
                                               ].apply(lambda x: x[str(y) + "_x"]**2 + x[str(y) + "_y"]**2 ,axis = 1 ) )
                            for y in loc_df[loc_df['status_0'] == 1].index], axis = 1))
het_df['min'] = het_df.min(axis =1)

new_df = loc_df.copy()
new_df['status_1'] = np.where(new_df.index.isin(het_df[het_df['min'] < 0.03].index), 1,
                             new_df['status_0'])
new_df[new_df['status_1']==1].shape

Wall time: 12.5 s


(1089, 5)